In [1]:
import os
from dotenv import load_dotenv
import asyncio

from autogen_agentchat.agents import AssistantAgent,UserProxyAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination

from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


# Load environment variables from a .env file
load_dotenv("/home/azureuser/azure-ai-agent-workshop/.env")

True

In [2]:
model_client=AzureOpenAIChatCompletionClient(
            model=os.getenv("AZURE_OPENAI_MODEL"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_KEY"),
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            api_version=os.getenv("AZUER_OPENAI_API_VERSION")
        )

In [3]:
# Define a function to simulate user feedback
def custom_input(task):
    return "上海正在下雨,天气很糟糕。"

In [4]:
# define Agent
weather_agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    description="一个和用户交互的智能体。"
)

# define Agent
user_proxy = UserProxyAgent(
    name="user_proxy",
    input_func=custom_input
)

In [5]:
# Define a termination condition. End the conversation if a specific text is mentioned.
text_termination = TextMentionTermination("<stop autogenworkshop>")
# Define a termination condition: stop the task after 5 messages
max_message_termination = MaxMessageTermination(4)
# Use the `|` operator to combine termination conditions, stopping the task if any condition is met
termination = text_termination | max_message_termination

In [6]:
# Define Team with type set to RoundRobinGroupChat
agent_team = RoundRobinGroupChat(participants=[weather_agent,user_proxy], termination_condition=termination, max_turns=None)


In [7]:
# Run the team and use the official Console tool to output in the proper format
stream = agent_team.run_stream(task="上海的天气如何?")
await Console(stream)

---------- user ----------
上海的天气如何?


---------- weather_agent ----------
抱歉，我无法提供实时数据或天气信息。不过，您可以通过天气预报网站或应用程序获取上海的最新天气信息。需要我帮您做其他事情吗？
---------- user_proxy ----------
上海正在下雨,天气很糟糕。
---------- weather_agent ----------
听起来上海的天气确实不太理想。下雨天可以考虑在室内活动，比如阅读、看电影或做些烹饪。需要我提供一些室内活动的建议吗？


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='上海的天气如何?', type='TextMessage'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=41, completion_tokens=38), content='抱歉，我无法提供实时数据或天气信息。不过，您可以通过天气预报网站或应用程序获取上海的最新天气信息。需要我帮您做其他事情吗？', type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, request_id='db9383a4-d8dd-4cfa-9a31-2398cde7f0af', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, content='上海正在下雨,天气很糟糕。', type='TextMessage'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=103, completion_tokens=45), content='听起来上海的天气确实不太理想。下雨天可以考虑在室内活动，比如阅读、看电影或做些烹饪。需要我提供一些室内活动的建议吗？', type='TextMessage')], stop_reason='Maximum number of messages 4 reached, current message count: 4')